In [ ]:
import wandb

def get_run_ids_with_filter(entity, project, filter_dict):
    """
    Retrieve the run IDs for all runs in a project that match the given filter.

    Args:
        entity (str): The W&B entity (username or team).
        project (str): The name of the W&B project.
        filter_dict (dict): A dictionary defining the filter criteria.
            For example: {"config.learning_rate": 0.001}
    
    Returns:
        list: A list of run IDs that match the filter.
    """
    api = wandb.Api()
    runs = api.runs(f"{entity}/{project}", filters=filter_dict)
    run_ids = [run.id for run in runs]
    run_names = [run.name for run in runs]
    return run_ids, run_names

def pull_run_data(run_id, entity="confopt-team", project="ConfoptAutoML25"):
    """
    Pull data from a W&B run given the entity, project, and run_id.
    
    Args:
        entity (str): The W&B entity (username or team).
        project (str): The name of the W&B project.
        run_id (str): The run identifier.
    
    Returns:
        dict: A dictionary containing the run summary, config, and history (as a pandas DataFrame).
    """
    api = wandb.Api()
    run_path = f"{entity}/{project}/{run_id}"
    try:
        run = api.run(run_path)
    except Exception as e:
        print(f"Error retrieving run: {e}")
        return None

    # Retrieve summary metrics (as a dict)
    summary = run.summary._json_dict

    # Retrieve configuration used for the run
    config = run.config

    # Retrieve run history (logged metrics) as a pandas DataFrame
    history = run.history(pandas=True)

    return {"summary": summary, "config": config, "history": history, "run_name": run.name}

In [ ]:
# Replace these with your actual values
entity = "confopt-team"
project = "ConfoptAutoML25"

# Define your filter. For example, filtering runs with a specific learning rate.
filter_dict = {
    # "config.benchmark": "deep-regular",
    # "config.dataset": "cifar10",
    # "config.sampler_type": "darts",
    "state": "running",
}

# Retrieve the run IDs matching the filter
matching_run_ids, matching_run_names = get_run_ids_with_filter(entity, project, filter_dict)
print(f"Matching run IDs: {matching_run_ids}")
print(f"Matching run names: {matching_run_names}")


In [ ]:
datas = dict()

for run_id, run_name in zip(matching_run_ids, matching_run_names):
    data = pull_run_data(run_id)
    datas[run_id] = data


In [ ]:
datas.keys()

In [ ]:
for run_id, data in datas.items():
    run_name = data["run_name"]
    summary = data["summary"]
    config = data["config"]

    runtime = summary["_runtime"]
    epoch = summary["_step"]
    time_per_epoch = runtime / epoch

    total_epochs = config["trainer"]["epochs"]
    time_left = (total_epochs - epoch) * time_per_epoch
    will_finish = (total_epochs - epoch) * time_per_epoch + runtime < 24 * 3600

    print(f"Run ID: {run_id}, time per epoch: {time_per_epoch:7.2f} \truntime: {runtime:8.02f}\tepoch: {epoch:3}/{total_epochs:3} \ttime left: {time_left/3600:6.2f} \twill finish: {will_finish} \trun_name: {run_name}")